In [1]:
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/dsrscientist/DSData/master/Employees.csv")

In [3]:
df.head()

,NAME,JOB TITLE,DEPARTMENT,EMPLOYEE ANNUAL SALARY,ESTIMATED ANNUAL SALARY MINUS FURLOUGHS
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,$81000.00,$73862.00
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,$74628.00,$74628.00
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,FLEET MANAGEMNT,$77280.00,$70174.00
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,$96276.00,$96276.00
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,WATER MGMNT,$84032.00,$76627.00


In [4]:
df.shape

(34218, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34218 entries, 0 to 34217
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   NAME                                     34218 non-null  object
 1   JOB TITLE                                34218 non-null  object
 2   DEPARTMENT                               34218 non-null  object
 3   EMPLOYEE ANNUAL SALARY                   34218 non-null  object
 4   ESTIMATED ANNUAL SALARY MINUS FURLOUGHS  34218 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


In [6]:
df.columns

Index(['NAME', 'JOB TITLE', 'DEPARTMENT', 'EMPLOYEE ANNUAL SALARY',
       'ESTIMATED ANNUAL SALARY MINUS FURLOUGHS'],
      dtype='object')

In [7]:
for col_name in df.columns:
    if df[col_name].dtypes == 'object':
        unique_val = len(df[col_name].unique())
    print("feature {col_name} has {unique_val} unique categories".format(col_name=col_name,unique_val=unique_val))

feature NAME has 33908 unique categories
feature JOB TITLE has 1124 unique categories
feature DEPARTMENT has 39 unique categories
feature EMPLOYEE ANNUAL SALARY has 1028 unique categories
feature ESTIMATED ANNUAL SALARY MINUS FURLOUGHS has 1088 unique categories


In [8]:
EMPLOYEE_ANNUAL_SALARY = df['EMPLOYEE ANNUAL SALARY'].apply(lambda x: x.replace('$',''))

In [9]:
ESTIMATED_ANN_SAL_MINUS_FURLOUGHS = df['ESTIMATED ANNUAL SALARY MINUS FURLOUGHS'].apply(lambda x: x.replace('$',''))

In [10]:
inputs = df.drop(['ESTIMATED ANNUAL SALARY MINUS FURLOUGHS','EMPLOYEE ANNUAL SALARY'],axis=1)

In [11]:
inputs.head()

,NAME,JOB TITLE,DEPARTMENT
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,FLEET MANAGEMNT
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,WATER MGMNT


In [12]:
df2 = pd.concat([inputs,ESTIMATED_ANN_SAL_MINUS_FURLOUGHS,EMPLOYEE_ANNUAL_SALARY],axis='columns')

In [13]:
df2.head()

,NAME,JOB TITLE,DEPARTMENT,ESTIMATED ANNUAL SALARY MINUS FURLOUGHS,EMPLOYEE ANNUAL SALARY
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,73862.00,81000.00
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,74628.00,74628.00
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,FLEET MANAGEMNT,70174.00,77280.00
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,96276.00,96276.00
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,WATER MGMNT,76627.00,84032.00


In [14]:
df2["ESTIMATED ANNUAL SALARY MINUS FURLOUGHS"] = df2["ESTIMATED ANNUAL SALARY MINUS FURLOUGHS"].astype('float')

In [15]:
df2["EMPLOYEE ANNUAL SALARY"] = df2["EMPLOYEE ANNUAL SALARY"].astype('float')

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34218 entries, 0 to 34217
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   NAME                                     34218 non-null  object 
 1   JOB TITLE                                34218 non-null  object 
 2   DEPARTMENT                               34218 non-null  object 
 3   ESTIMATED ANNUAL SALARY MINUS FURLOUGHS  34218 non-null  float64
 4   EMPLOYEE ANNUAL SALARY                   34218 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.3+ MB


# Building the model

In [17]:
# Features

x = df2.copy(deep=True)
x.drop(["NAME","ESTIMATED ANNUAL SALARY MINUS FURLOUGHS"], axis = 1, inplace = True)
x = pd.get_dummies(x)
x.head()

,EMPLOYEE ANNUAL SALARY,JOB TITLE_A/MGR COM SVC-ELECTIONS,JOB TITLE_A/MGR OF MIS-ELECTIONS,JOB TITLE_A/MGR WAREHOUSE-ELECTIONS,JOB TITLE_A/SUPRV REDISTRICTING,JOB TITLE_ACCIDENT ADJUSTER,JOB TITLE_ACCOUNTANT I,JOB TITLE_ACCOUNTANT II,JOB TITLE_ACCOUNTANT III,JOB TITLE_ACCOUNTANT IV,...,DEPARTMENT_OEMC,DEPARTMENT_POLICE,DEPARTMENT_POLICE BOARD,DEPARTMENT_PROCUREMENT,DEPARTMENT_PUBLIC LIBRARY,DEPARTMENT_REVENUE,DEPARTMENT_STREETS & SAN,DEPARTMENT_TRANSPORTN,DEPARTMENT_TREASURER,DEPARTMENT_WATER MGMNT
0,81000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,74628.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,77280.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,96276.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,84032.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
y = df2['ESTIMATED ANNUAL SALARY MINUS FURLOUGHS']

In [19]:
x.shape

(34218, 1164)

In [20]:
y.shape

(34218,)

In [21]:
# Splitting the data

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

# Random Forest Classifier

In [24]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train,y_train)

pred_rf = rf.predict(X_test)

print("Random Forest test set Performance: ")
print("-------------------------------------")
print(accuracy_score(y_test,pred_rf))
print(confusion_matrix(y_test,pred_rf))

Random Forest test set Performance: 
-------------------------------------
0.9203682057276447
[[53  0  0 ...  0  0  0]
 [ 0 14  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  2  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


# K NEAREST NEIGHBORS

In [26]:
kn = KNeighborsClassifier(n_neighbors = 3)
kn.fit(X_train, y_train)

pred_kn = kn.predict(X_test)

print("Knn test set Performance: ")
print("---------------------------")
print(accuracy_score(y_test,pred_kn))
print(confusion_matrix(y_test,pred_kn))

Knn test set Performance: 
---------------------------
0.9758912916423145
[[53  0  0 ...  0  0  0]
 [ 0 14  0 ...  0  0  0]
 [ 0  0  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  1  0]]
